In [6]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense # for creating regular densely-connected NN layers.
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Data manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Sklearn
import sklearn # for model evaluation
from sklearn.model_selection import train_test_split # for splitting data into train and test samples
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Erstellung eigener Aktivierungsfunktion
from keras import backend as K
import os
import scipy
from sklearn.mixture import GaussianMixture

# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
from keras.models import Sequential, load_model # add load_model to the import

In [7]:
class gmm_model():

    def __init__(self):
        self.test_size = 0.35
        self.p = 0.3

    def import_data(self, sample_size = 11500):
        csv_files = []
        for filename in os.listdir():
            if filename == "Datasets":
                for csv_file in os.listdir(filename):
                    if csv_file.endswith('.csv'):
                        csv_files.append(csv_file)
        raw_data = pd.read_csv(os.path.join("Datasets", csv_files[0]))
        raw_data.drop(["Time"], axis=1, inplace=True)
        raw_data.drop(["Amount"], axis=1, inplace=True)

        # Auswahl von 4000 zufälligen Zeilen
        raw_data = raw_data.sample(n=sample_size, random_state=42)

        Y = raw_data["Class"]
        X = raw_data.drop(["Class"], axis=1)

        return X, Y, raw_data

    def gen_gmm(self, data):
        data.drop(["Class"], axis=1, inplace=True)
        # erstellen GMM über gesamtes
        gmm = GaussianMixture(n_components=3, covariance_type="diag")
        gmm.fit(data)
        return gmm

    def gen_miss_values(self, data, p):
        shape = data.shape
        new_df = data.copy().astype(np.float64)
        missing = np.random.binomial(1, self.p, shape)
        data[missing.astype('bool')] = np.nan

        return data

    def train_test_split(self, X, Y, test_size):
        # train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=self.test_size, random_state=42)
        return X_train, X_test, y_train, y_test

    def gmm_activation_smieja(self, data_missing, gmm, model):
        w = model.layers[0].get_weights()[0]  # Gewichte von erster Schicht
        b = model.layers[0].get_weights()[1][:, np.newaxis]  # Bias erste Schicht
        output_list = []
        # man muss nur durch spalten iterieren
        for s in range(0, len(pd.DataFrame(w).columns)):
            for i in range(0, len(data_missing)):  # iteriere DF mit NaN-Values
                list_nan = np.array(data_missing.iloc[i])

                result = 0
                for ii in range(0, gmm.n_components):  # iteriere GMM mit n_komp
                    list_gmm_cov = gmm.covariances_[ii]
                    list_gmm_mean = gmm.means_[ii]
                    p_i = gmm.weights_[ii]

                    # Erstellen von GMM-Vektoren
                    # create cov_gmm: in gmm_covariances_vektor werte 0 setzen die in der jeweiligen Spalte von data_missing nicht NaN sind
                    global s_i
                    s_i = []
                    for y in range(len(list_gmm_cov)):
                        if np.isnan(list_nan[y]):
                            s_i.append(list_gmm_cov[y])
                        else:
                            s_i.append(0)

                    # create mean_gmm: in gmm_mean_vektor werden alle festen werte aufgenommen aus data_missing und NaN-Werte durch list_gmm_mean ersetzt
                    m_i = []
                    for y in range(len(list_gmm_mean)):
                        if not np.isnan(list_nan[y]):
                            m_i.append(list_nan[y])
                        else:
                            m_i.append(list_gmm_mean[y])

                    # Berechnen ReLUw,b(F)
                    ft = p_i * np.sqrt(np.transpose(np.array(w[:, s]) * w[:, s]) @ s_i)
                    z = np.transpose(np.array(w[:, s])) @ m_i + b[s]  # Zähler für NR w
                    n = np.sqrt(np.transpose(np.array(w[:, s]) * w[:, s]) @ s_i)  # Nenner für NR w
                    nr_w = z / n  # Input in NR
                    nr = 1 / np.sqrt(2 * np.pi) * np.exp(-(np.square(nr_w) / 2)) + (nr_w / 2) * (
                                1 + scipy.special.erf(nr_w / np.sqrt(2)))  # NR-Funktion
                    ReLUwb = ft * nr  # Summe aus i-GMM-Komponenten
                    # print(f'ReLUwb:{ReLUwb:} ft:{ft:} z:{z:} n:{n:} nr_w:{nr_w:} nr:{nr:}')
                    result += ReLUwb
                    # print(f'resutl{result}')
            output_list.append(result)
        return output_list

    def model_01(self, X_train, X_test, y_train, y_test):
        model = tf.keras.Sequential(name="Model")
        model.add(tf.keras.layers.Dense(units=32, activation=tf.nn.relu, input_shape=[len(X_train.columns)]))
        model.add(tf.keras.layers.Dense(units=64, activation=tf.nn.relu))
        model.add(tf.keras.layers.Dense(units=128, activation=tf.nn.relu))
        model.add(tf.keras.layers.Dense(units=2, activation=tf.nn.sigmoid))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=5)

        # Fit model with early stopping
        model.fit(X_train, tf.keras.utils.to_categorical(y_train), epochs=50,
                  validation_data=(X_test, tf.keras.utils.to_categorical(y_test)),
                  callbacks=[early_stop])
        return model

    def model_02(self, X_train, X_test, y_train, y_test):
        model = tf.keras.Sequential(name="Model")
        model.add(tf.keras.layers.Dense(units=32, activation=tf.nn.relu, input_shape=[len(X_train.columns)]))
        model.add(tf.keras.layers.Dense(units=64, activation=tf.nn.relu))
        model.add(tf.keras.layers.Dense(units=128, activation=tf.nn.relu))
        model.add(tf.keras.layers.Dense(units=256, activation=tf.nn.relu))
        model.add(tf.keras.layers.Dense(units=2, activation=tf.nn.sigmoid))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=5)

        # Fit model with early stopping
        model.fit(X_train, tf.keras.utils.to_categorical(y_train), epochs=50,
                  validation_data=(X_test, tf.keras.utils.to_categorical(y_test)),
                  callbacks=[early_stop])
        return model

    def model_03(self, X_train, X_test, y_train, y_test):
        model = tf.keras.Sequential(name="Model")
        model.add(tf.keras.layers.Dense(units=100, activation=tf.nn.relu, input_shape=[len(X_train.columns)]))
        model.add(tf.keras.layers.Dense(units=400, activation=tf.nn.relu))
        model.add(tf.keras.layers.Dense(units=600, activation=tf.nn.relu))
        model.add(tf.keras.layers.Dense(units=2, activation=tf.nn.sigmoid))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=5)

        # Fit model with early stopping
        model.fit(X_train, tf.keras.utils.to_categorical(y_train), epochs=50,
                  validation_data=(X_test, tf.keras.utils.to_categorical(y_test)),
                  callbacks=[early_stop])
        return model

    def predict_model(self, X_test_m, model):
        if len(model.layers) == 4:
            predictions = []
            for index, row in X_test_m.iterrows():
                row_df = pd.DataFrame([row])
                row_df.columns = [f"x{i + 1}" for i in range(len(row_df.columns))]

                gmm_output = self.gmm_activation_smieja(row_df, gmm, model)

                input_data = np.array(gmm_output).reshape((1, len(gmm_output)))
                get_layer_output = K.function([model.layers[1].input], [model.layers[1].output])
                output = get_layer_output([input_data])[0]
                get_layer_output = K.function([model.layers[2].input], [model.layers[2].output])
                output = get_layer_output([output])[0]
                get_layer_output = K.function([model.layers[3].input], [model.layers[3].output])
                output = get_layer_output([output])[0]
                index_of_max = np.argmax(output)
                print(f'output:{output} index_of_max:{index_of_max}')
                predictions.append(index_of_max)

            return predictions

        elif len(model.layers) == 5:
            predictions = []
            for index, row in X_test_m.iterrows():
                row_df = pd.DataFrame([row])
                row_df.columns = [f"x{i + 1}" for i in range(len(row_df.columns))]

                gmm_output = self.gmm_activation_smieja(row_df, gmm, model)

                input_data = np.array(gmm_output).reshape((1, len(gmm_output)))
                get_layer_output = K.function([model.layers[1].input], [model.layers[1].output])
                output = get_layer_output([input_data])[0]
                get_layer_output = K.function([model.layers[2].input], [model.layers[2].output])
                output = get_layer_output([output])[0]
                get_layer_output = K.function([model.layers[3].input], [model.layers[3].output])
                output = get_layer_output([output])[0]
                get_layer_output = K.function([model.layers[4].input], [model.layers[4].output])
                output = get_layer_output([output])[0]
                index_of_max = np.argmax(output)
                print(f'output:{output} index_of_max:{index_of_max}')
                predictions.append(index_of_max)

            return predictions

        else:
            print("Something wrong")

    def evaluate(self, y_test, y_gmm_nan):
        y_true = y_test
        y_pred = y_gmm_nan
        conf_matrix = confusion_matrix(y_true, y_pred)

        # Compute the accuracy
        accuracy = accuracy_score(y_true, y_pred)

        # Compute the precision
        precision_scores = precision_score(y_true, y_pred, labels=range(len(conf_matrix)), average=None)

        # Compute the recall
        recall = recall_score(y_true, y_pred, labels=range(len(conf_matrix)), average=None)

        # Compute the F1-score
        f1 = f1_score(y_true, y_pred, labels=range(len(conf_matrix)), average=None)

        # Compute the average accuracy
        avg_accuracy = sum(precision_scores) / len(conf_matrix)

        # Compute the predicted probabilities
        y_scores = np.random.rand(len(y_true), len(np.unique(y_true)))

        # Compute the AUC for each class
        n_classes = conf_matrix.shape[0]
        auc_list = []
        for i in range(n_classes):
            auc_list.append(roc_auc_score(y_true == i, y_scores[:, i]))

        # Define the result dictionary
        result = {
            "confusion_matrix": conf_matrix,
            "accuracy": accuracy,
            "precision": precision_scores,
            "recall": recall,
            "f1_score": f1,
            "avg_accuracy": avg_accuracy,
            "auc": auc_list
        }
        return result

    def save_txt(self, filename, evaluate):
        with open(filename, 'w') as f:
            f.write(str(evaluate))


# Model_01-_03 Test
# p = 0.3,  0.6, 0,9

# List of models to test
models = [("Model_01", "predictions_D0_01_"), ("Model_02", "predictions_D0_02_"), ("Model_03", "predictions_D0_03_")]

# List of missing values percentages to test
p_values = [0.3, 0.6, 0.9]

for model_name, output_prefix in models:
    # Initialize model and prepare data
    model = gmm_model()
    X_raw, Y_raw, data_raw = model.import_data()
    X_train, X_test, y_train, y_test = model.train_test_split(X_raw, Y_raw, 0.33)

    # Load model and create gmm
    if model_name == "Model_01":
        trained_model = load_model('model_D0_01.h5')  # load the model instead of creating it
    elif model_name == "Model_02":
        trained_model = load_model('model_D0_02.h5')  # load the model instead of creating it
    elif model_name == "Model_03":
        trained_model = load_model('model_D0_03.h5')  # load the model instead of creating it


    gmm = model.gen_gmm(data_raw)

    for p in p_values:
        # Generate missing values data and predict values
        data_miss = model.gen_miss_values(X_test, p)
        predictions = model.predict_model(data_miss, trained_model)

        # Evaluate model and save results to file
        evaluate = model.evaluate(y_test, predictions)
        output_filename = output_prefix + str(p).replace(".", "_")
        model.save_txt(output_filename, evaluate)

output:[[1.0000000e+00 2.4748539e-15]] index_of_max:0
output:[[1.000000e+00 3.247894e-12]] index_of_max:0
output:[[1.000000e+00 2.320531e-09]] index_of_max:0
output:[[1.000000e+00 7.474428e-12]] index_of_max:0
output:[[1.0000000e+00 3.5371595e-05]] index_of_max:0
output:[[1.0000000e+00 8.1571214e-13]] index_of_max:0
output:[[1.0000000e+00 5.2184177e-09]] index_of_max:0
output:[[1.000000e+00 3.699509e-09]] index_of_max:0
output:[[1.0000000e+00 1.9276271e-08]] index_of_max:0
output:[[1.0000000e+00 2.0278594e-07]] index_of_max:0
output:[[1.0000000e+00 5.8161627e-12]] index_of_max:0
output:[[1.0000000e+00 4.5008762e-12]] index_of_max:0
output:[[1.0000000e+00 3.0851754e-10]] index_of_max:0
output:[[1.0000000e+00 1.1121974e-09]] index_of_max:0
output:[[1.00000e+00 6.05332e-09]] index_of_max:0
output:[[1.000000e+00 7.596934e-07]] index_of_max:0
output:[[0.99962646 0.12253229]] index_of_max:0
output:[[1.0000000e+00 7.5299006e-13]] index_of_max:0
output:[[1.000000e+00 4.598222e-10]] index_of_ma

output:[[1.0000000e+00 4.6061937e-06]] index_of_max:0
output:[[1.0000000e+00 4.0590667e-08]] index_of_max:0
output:[[1.0000000e+00 1.3971004e-10]] index_of_max:0
output:[[1.00000e+00 4.06528e-12]] index_of_max:0
output:[[1.0000000e+00 4.0918536e-12]] index_of_max:0
output:[[1.000000e+00 4.552959e-08]] index_of_max:0
output:[[1.0000000e+00 2.5080507e-08]] index_of_max:0
output:[[1.0000000e+00 3.3408893e-09]] index_of_max:0
output:[[1.000000e+00 5.959713e-12]] index_of_max:0
output:[[1.0000000e+00 4.2949685e-13]] index_of_max:0
output:[[1.0000000e+00 1.3245899e-08]] index_of_max:0
output:[[1.0000000e+00 3.4775618e-09]] index_of_max:0
output:[[1.0000000e+00 1.0879901e-14]] index_of_max:0
output:[[1.0000000e+00 2.1792353e-06]] index_of_max:0
output:[[1.0000000e+00 2.6340301e-12]] index_of_max:0
output:[[1.000000e+00 9.692603e-05]] index_of_max:0
output:[[1.000000e+00 5.694206e-10]] index_of_max:0
output:[[1.00000e+00 1.78953e-06]] index_of_max:0
output:[[1.0000000e+00 1.2004549e-16]] index

output:[[1.0000000e+00 1.0092812e-09]] index_of_max:0
output:[[1.0000000e+00 1.9381378e-08]] index_of_max:0
output:[[1.000000e+00 4.253717e-09]] index_of_max:0
output:[[1.000000e+00 3.338163e-09]] index_of_max:0
output:[[1.0000000e+00 1.5721824e-12]] index_of_max:0
output:[[1.00000000e+00 1.08752365e-05]] index_of_max:0
output:[[1.000000e+00 2.232311e-08]] index_of_max:0
output:[[1.000000e+00 4.700171e-05]] index_of_max:0
output:[[1.         0.00117878]] index_of_max:0
output:[[1.0000000e+00 8.0061935e-15]] index_of_max:0
output:[[1.0000000e+00 2.3296312e-09]] index_of_max:0
output:[[1.000000e+00 4.493275e-05]] index_of_max:0
output:[[1.0000000e+00 1.8339591e-20]] index_of_max:0
output:[[1.         0.00158633]] index_of_max:0
output:[[1.0000000e+00 1.6943481e-11]] index_of_max:0
output:[[1.0000000e+00 2.9990394e-10]] index_of_max:0
output:[[1.0000000e+00 3.2830654e-12]] index_of_max:0
output:[[1.0000000e+00 1.7325587e-10]] index_of_max:0
output:[[1.0000000e+00 1.9955782e-04]] index_of_

output:[[1.0000000e+00 1.9192247e-07]] index_of_max:0
output:[[1.0000000e+00 1.5324945e-08]] index_of_max:0
output:[[1.0000000e+00 5.5822596e-07]] index_of_max:0
output:[[1.0000000e+00 4.0141244e-13]] index_of_max:0
output:[[1.0000000e+00 2.3042344e-09]] index_of_max:0
output:[[1.0000000e+00 2.4291952e-11]] index_of_max:0
output:[[1.0000000e+00 2.9252185e-09]] index_of_max:0
output:[[1.         0.00331406]] index_of_max:0
output:[[1.000000e+00 7.928915e-08]] index_of_max:0
output:[[1.0000000e+00 4.2168363e-11]] index_of_max:0
output:[[1.0000000e+00 1.1712016e-10]] index_of_max:0
output:[[1.0000000e+00 1.6916415e-08]] index_of_max:0
output:[[1.0000000e+00 6.7703647e-09]] index_of_max:0
output:[[1.000000e+00 9.723615e-07]] index_of_max:0
output:[[1.0000000e+00 4.2189236e-05]] index_of_max:0
output:[[1.0000000e+00 2.0394155e-12]] index_of_max:0
output:[[1.00000000e+00 1.40804595e-08]] index_of_max:0
output:[[1.0000000e+00 1.2933443e-10]] index_of_max:0
output:[[1.0000000e+00 5.3622387e-09

output:[[1.0000000e+00 1.3682558e-09]] index_of_max:0
output:[[1.000000e+00 8.140585e-09]] index_of_max:0
output:[[1.0000000e+00 2.2958374e-13]] index_of_max:0
output:[[1.000000e+00 8.611613e-09]] index_of_max:0
output:[[1.0000000e+00 4.0199017e-09]] index_of_max:0
output:[[1.0000000e+00 1.1073665e-06]] index_of_max:0
output:[[1.         0.00706406]] index_of_max:0
output:[[1.0000000e+00 2.3594893e-07]] index_of_max:0
output:[[1.0000000e+00 1.6637416e-12]] index_of_max:0
output:[[1.000000e+00 6.687349e-07]] index_of_max:0
output:[[1.0000000e+00 5.5279707e-13]] index_of_max:0
output:[[1.000000e+00 3.333851e-08]] index_of_max:0
output:[[1.0000000e+00 4.2892478e-16]] index_of_max:0
output:[[1.0000000e+00 1.4524642e-13]] index_of_max:0
output:[[1.0000000e+00 2.0555944e-09]] index_of_max:0
output:[[1.000000e+00 6.178106e-09]] index_of_max:0
output:[[1.0000000e+00 2.5751712e-04]] index_of_max:0
output:[[1.000000e+00 2.119614e-07]] index_of_max:0
output:[[1.0000000e+00 7.7194836e-14]] index_o

...

output:[[9.999969e-01 7.652284e-14]] index_of_max:0
output:[[9.9998963e-01 1.4565903e-12]] index_of_max:0
output:[[9.9999380e-01 3.2602995e-13]] index_of_max:0
output:[[9.9999410e-01 3.3343955e-13]] index_of_max:0
output:[[9.9998230e-01 3.9961207e-12]] index_of_max:0
output:[[9.9999487e-01 1.9932053e-13]] index_of_max:0
output:[[9.9999994e-01 5.7174616e-18]] index_of_max:0
output:[[9.9999923e-01 2.0137234e-15]] index_of_max:0
output:[[9.9999195e-01 6.5805239e-13]] index_of_max:0
output:[[9.9998558e-01 2.8881415e-12]] index_of_max:0
output:[[9.9999577e-01 1.7523872e-13]] index_of_max:0
output:[[9.999947e-01 2.235629e-13]] index_of_max:0
output:[[9.9999684e-01 7.3701848e-14]] index_of_max:0
output:[[9.999870e-01 1.833407e-12]] index_of_max:0
output:[[9.9999785e-01 3.3121770e-14]] index_of_max:0
output:[[9.9998802e-01 1.7330395e-12]] index_of_max:0
output:[[9.9997520e-01 8.5625465e-12]] index_of_max:0
output:[[9.9999815e-01 2.1753836e-14]] index_of_max:0
output:[[9.9999779e-01 2.5701555e-

output:[[9.9999917e-01 4.9657383e-15]] index_of_max:0
output:[[9.9998772e-01 1.4207765e-12]] index_of_max:0
output:[[9.9999076e-01 7.8036476e-13]] index_of_max:0
output:[[9.9999386e-01 4.0170577e-13]] index_of_max:0
output:[[9.9999285e-01 4.9014449e-13]] index_of_max:0
output:[[9.9998820e-01 1.8456168e-12]] index_of_max:0
output:[[9.9999362e-01 3.8143672e-13]] index_of_max:0
output:[[9.9999833e-01 1.7440202e-14]] index_of_max:0
output:[[9.9995536e-01 3.3491737e-11]] index_of_max:0
output:[[9.9999410e-01 3.1113317e-13]] index_of_max:0
output:[[9.9999952e-01 7.7582373e-16]] index_of_max:0
output:[[9.9998569e-01 2.4495215e-12]] index_of_max:0
output:[[9.9999034e-01 1.0155168e-12]] index_of_max:0
output:[[9.99996126e-01 1.05965604e-13]] index_of_max:0
output:[[9.9999964e-01 3.8202078e-16]] index_of_max:0
output:[[9.9999696e-01 8.1525600e-14]] index_of_max:0
output:[[9.9998772e-01 1.8369074e-12]] index_of_max:0
output:[[9.999986e-01 6.921413e-15]] index_of_max:0
output:[[9.9999577e-01 1.225

output:[[9.9999905e-01 4.3960616e-15]] index_of_max:0
output:[[9.9999106e-01 9.2437939e-13]] index_of_max:0
output:[[9.9999475e-01 2.3248016e-13]] index_of_max:0
output:[[9.999970e-01 7.622407e-14]] index_of_max:0
output:[[9.9999660e-01 8.6796625e-14]] index_of_max:0
output:[[9.9998957e-01 1.1141045e-12]] index_of_max:0
output:[[9.999995e-01 8.017926e-16]] index_of_max:0
output:[[9.999936e-01 3.816827e-13]] index_of_max:0
output:[[9.9999970e-01 2.6628278e-16]] index_of_max:0
output:[[9.9999988e-01 2.3150841e-17]] index_of_max:0
output:[[9.9999154e-01 7.3134608e-13]] index_of_max:0
output:[[9.999995e-01 5.234049e-16]] index_of_max:0
output:[[9.9998933e-01 1.5780557e-12]] index_of_max:0
output:[[9.9998260e-01 4.7617587e-12]] index_of_max:0
output:[[9.9999833e-01 1.2543104e-14]] index_of_max:0
output:[[9.999909e-01 9.345713e-13]] index_of_max:0
output:[[9.9999690e-01 6.9274786e-14]] index_of_max:0
output:[[9.9999058e-01 1.4341447e-12]] index_of_max:0
output:[[9.9999326e-01 6.6021098e-13]]

output:[[9.9999911e-01 2.3182944e-15]] index_of_max:0
output:[[9.9999684e-01 7.3264833e-14]] index_of_max:0
output:[[9.9998969e-01 1.0372599e-12]] index_of_max:0
output:[[9.9999499e-01 2.3145898e-13]] index_of_max:0
output:[[1.000000e+00 5.582796e-20]] index_of_max:0


C:\Users\Ivan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
